# Network measures

### Local structures

**Indegree**
This is mostly a function of how Wikipedians revised the document and should largely be uniform across pages. The large values are likely pages with 'lists' of links.

**Outdegree**
This is 1st-order measure of an idea's influence.

### Mesoscale structures

**Clustering**
These look equally clustered among the topics.

**Centrality**
This reveals the distribution of sources of ideas within a field.

**Path lengths**

**Rich-club coefficient**

**Modularity**

**Controllability**
This is an nth-order measure of influence.

**Observability**
This is an nth-order measure of the inverse of influence.

**Coreness**
It seems that the more focused a topic is on a subtopic, the stronger the coreness. For example, genetics is heavily focused on DNA, and so it has high coreness. At the same time, in the field of economics, the concept of "economics" has high degree. Yet, it has low coreness because the field itself is heterogeneous, with major subfields such as "macroeconomics" and "microeconomics".

**Characteristic path length**
I'm not sure what path length reveals. Perhaps it is a measure of the heterogeneity in research? It describes how far one idea is to another, topologically. Cognitive science and earth science have ideas that are far away.

# Load networks

In [ ]:
%reload_ext autoreload
%autoreload 2
import os,sys
sys.path.insert(1, os.path.join(sys.path[0], '..', 'module'))
import wiki
import numpy as np
import pandas as pd
import networkx as nx

In [ ]:
path_networks = '/Users/harangju/Developer/data/wiki/graphs/'

In [ ]:
topics = ['anatomy', 'biochemistry', 'cognitive science', 'evolutionary biology',
          'genetics', 'immunology', 'molecular biology', 'chemistry', 'biophysics',
          'energy', 'optics', 'earth science', 'geology', 'meteorology',
          'philosophy of language', 'philosophy of law', 'philosophy of mind',
          'philosophy of science', 'economics', 'accounting', 'education',
          'linguistics', 'law', 'psychology', 'sociology', 'electronics',
          'software engineering', 'robotics',
          'calculus', 'geometry', 'abstract algebra',
          'Boolean algebra', 'commutative algebra', 'group theory', 'linear algebra',
          'number theory', 'dynamical systems and differential equations']

In [ ]:
networks = {}
for topic in topics:
    print(topic, end=' ')
    networks[topic] = wiki.Net()
    networks[topic].load_graph(path_networks+'dated/'+topic+'.pickle')

In [ ]:
num_nulls = 10
null_targets = {}
for topic in topics:
    print(topic, end=' ')
    null_targets[topic] = []
    for i in range(num_nulls):
        network = wiki.Net()
        network.load_graph(path_networks+'null-target/'+topic+'-null-'+str(i)+'.pickle')
        null_targets[topic].append(network)

# Run analysis

**NOTE:** Skip section if loading stats.

In [ ]:
import bct
import pickle
from networkx.algorithms.cluster import clustering
from networkx.algorithms import betweenness_centrality
from networkx.convert_matrix import to_numpy_array

In [ ]:
measures = {'indegree': lambda g: [x[1] for x in g.in_degree],
            'outdegree': lambda g: [x[1] for x in g.out_degree],
            'clustering': lambda g: list(clustering(g).values()),
            'centrality': lambda g: list(betweenness_centrality(g).values()),
#             'path-length': lambda g: [y for x in list(nx.shortest_path_length(g))
#                                       for y in list(x[1].values())],
            'char-path-length': lambda g: bct.charpath(to_numpy_array(g))[0],
            'modularity': lambda g: g.graph['modularity'],
            'coreness': lambda g: g.graph['coreness_be']}

In [ ]:
df = pd.DataFrame(columns=['topic','measure','value'])
for topic, network in networks.items():
    print(topic, end=' ')
    df = pd.concat([df] +
                   [pd.DataFrame([[topic, measure, func(network.graph)]],
                                 columns=['topic','measure','value'])
                    for measure, func in {'coreness': measures['coreness']}.items()],#measures.items()],
                   ignore_index=True)

In [ ]:
for topic, null_networks in null_targets.items():
    print(topic, end=' ')
    for network in null_networks:
        df = pd.concat([df] + 
                       [pd.DataFrame([[topic, measure+'-null', func(network.graph)]],
                                     columns=['topic','measure','value'])
                        for measure, func in measures.items()],
                       ignore_index=True)

In [ ]:
df

# Save analysis

In [ ]:
path_analysis = '/Users/harangju/Developer/data/wiki/analysis/'

In [ ]:
pickle.dump(df, open(path_analysis + 'stats.pickle','wb'))

In [ ]:
df.topic = df.topic.astype('category')
df.measure = df.measure.astype('category')
df.dtypes

In [ ]:
df_expand = df.value\
              .apply(pd.Series)\
              .merge(df, left_index=True, right_index=True)\
              .drop('value', axis=1)\
              .melt(id_vars=['topic','measure'])\
              .drop('variable', axis=1)\
              .dropna()
df_expand

In [ ]:
pickle.dump(df_expand, open(path_analysis + 'stats_expand.pickle','wb'))

# Load analysis

In [ ]:
import pickle
import pandas as pd
path_analysis = '/Users/harangju/Developer/data/wiki/analysis/'
df = pickle.load(open(path_analysis+'stats.pickle', 'rb'))
df_expand = pickle.load(open(path_analysis+'stats_expand.pickle', 'rb'))
df.topic = df.topic.astype('object')
df.measure = df.measure.astype('object')
df_expand.topic = df_expand.topic.astype('object')
df_expand.measure = df_expand.measure.astype('object')

In [ ]:
df

In [ ]:
pd.unique(df.topic)

In [ ]:
pd.unique(df.measure)

In [ ]:
df.dtypes

In [ ]:
df_mean = df_expand\
    .groupby(['topic', 'measure'], as_index=False)\
    .mean()\
    .pivot(index='topic', columns='measure', values='value')\
    .reset_index()

# Plot

* nice plots [seaborn](https://seaborn.pydata.org/examples/index.html)
* interactive [Bokeh](https://bokeh.pydata.org/en/latest/docs/gallery.html#gallery)

In [ ]:
from ipywidgets import interact, widgets, Layout
import plotly
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from IPython.display import display
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
path_fig = '/Users/harangju/Box Sync/Research/my papers/wikipedia/results/'
save_fig = False

## Basic network statistics

In [ ]:
import os

path_plot = '1 network_stats'

if not os.path.exists(f"{path_fig}/{path_plot}"):
    os.mkdir(f"{path_fig}/{path_plot}")

### Individual

In [ ]:
for measure in ['indegree', 'outdegree', 'clustering', 'centrality']:#, 'path-length']:
    fig = px.box(df_expand[(df_expand.measure==measure) |
                           (df_expand.measure==measure+'-null-target')],
                 x='topic', y='value', color='measure')
    fig.update_layout(template='plotly_white',
                      yaxis_title=measure)
    fig.show()
    if save_fig:
        fig.write_image(f"{path_fig}/{path_plot}/{measure}.pdf")

In [ ]:
for measure in ['coreness', 'modularity', 'char-path-length']:
    fig = px.scatter(df_expand[(df_expand.measure==measure) |
                               (df_expand.measure==measure+'-null-target')],
                     x='topic', y='value', color='measure')
    fig.update_layout(template='plotly_white',
                      yaxis_title=measure)
    fig.show()
    if save_fig:
        fig.write_image(f"{path_fig}/{path_plot}/{measure}.pdf")

### Summary

In [ ]:
ranges = {'clustering': [0,0.3],
          'centrality': [0,0.04],
          'indegree': [0,10],
          'outdegree': [0,10]}
dticks = {'clustering': 0.1,
          'centrality': 0.01,
          'indegree': 1,
          'outdegree': 1}

In [ ]:
for measure in ['clustering','centrality','indegree','outdegree']:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_mean[measure],
                             y=df_mean[measure+'-null-target'],
                             mode='markers',
                             marker={'color': '#2A3F5F'},
                             showlegend=False))
    fig.add_trace(go.Scatter(x=ranges[measure], y=ranges[measure],
                             mode='lines',
                             line={'dash': 'dash',
                                   'color': '#2A3F5F'},
                             showlegend=False))
    fig.update_layout(template='plotly_white',
                      width=500, height=500,
                      title=measure,
                      xaxis={'title': 'real',
                             'range': ranges[measure],
                             'dtick': dticks[measure]},
                      yaxis={'title': 'null',
                             'range': ranges[measure],
                             'dtick': dticks[measure]})
    fig.show()
    if save_fig:
        fig.write_image(os.path.join(path_fig,path_plot,f"summary_{measure}.pdf"))

## Growing networks

In [ ]:
comm_t = pd.DataFrame()
for topic, network in networks.items():
    print(topic, end=' ')
    comm_t = pd.concat([comm_t] +
                       [pd.DataFrame([[topic,
                                       node,
                                       network.graph.nodes[node]['year'],
                                       network.graph.nodes[node]['community'],
                                       network.graph.nodes[node]['core_be'],
                                       network.graph.nodes[node]['core_rb'],
                                       1]],
                                     columns=['topic','node','year',
                                              'comm','core_be','core_rb',
                                              'count'])
                        for node in network.graph.nodes],
                       ignore_index=True)
comm_t = comm_t.merge(comm_t.groupby(['topic','comm'])['count'].sum(),
                      on=['topic','comm'],
                      suffixes=('','_topic_comm'))\
               .merge(comm_t.groupby(['topic','core_be'])['count'].sum(),
                      on=['topic','core_be'],
                      suffixes=('','_topic_core_be'))\
               .sort_values(by=['topic','year'])\
               .reset_index(drop=True)
comm_t['comm_count'] = comm_t.groupby(['topic','comm'])['count']\
                             .transform(pd.Series.cumsum)
comm_t['core_be_count'] = comm_t.groupby(['topic','core_be'])['count']\
                                .transform(pd.Series.cumsum)
comm_t['comm_frac'] = comm_t['comm_count']/comm_t['count_topic_comm']
comm_t['core_be_frac'] = comm_t['core_be_count']/comm_t['count_topic_core_be']
comm_t = comm_t.drop(['count','count_topic_comm','count_topic_core_be'], axis=1)

In [ ]:
comm_t

## Core-periphery

In [ ]:
import os

path_plot = '1 core_periphery'

if not os.path.exists(f"{path_fig}/{path_plot}"):
    os.mkdir(f"{path_fig}/{path_plot}")

### Static

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_mean['coreness'],
                         y=df_mean['coreness-null-target'],
                         mode='markers',
                         marker={'color': '#2A3F5F'},
                         showlegend=False,
                         hovertext=df_mean['topic']))
fig.add_trace(go.Scatter(x=[0,1], y=[0,1],
                         mode='lines',
                         line={'dash': 'dash',
                               'color': '#2A3F5F'},
                         showlegend=False))
fig.update_layout(template='plotly_white',
                  title='coreness',
                  width=500, height=500,
                  xaxis=dict(title='real',
                             range=[0,1]),
                  yaxis=dict(title='null',
                             range=[0,1],
                             scaleanchor='x',
                             scaleratio=1))
fig.show()
if save_fig:
    fig.write_image(f"{path_fig}/{path_plot}/coreness.pdf")

### Growth

##### Borgatti-Everett

In [ ]:
import os

if not os.path.exists(f"{path_fig}/{path_plot}/borgatti-everett/"):
    os.mkdir(f"{path_fig}/{path_plot}/borgatti-everett/")

In [ ]:
for topic in pd.unique(comm_t.topic):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=comm_t[(comm_t.topic==topic) &\
                                      (comm_t.core_be==0)]['year'],
                             y=comm_t[(comm_t.topic==topic) &\
                                      (comm_t.core_be==0)]['core_be_count'],
                             name='periphery'))
    fig.add_trace(go.Scatter(x=comm_t[(comm_t.topic==topic) &\
                                      (comm_t.core_be==1)]['year'],
                             y=comm_t[(comm_t.topic==topic) &\
                                      (comm_t.core_be==1)]['core_be_count'],
                             name='core'))
    fig.update_layout(template='plotly_white',
                      title_text=topic,
                      xaxis={'range': [0,2020]})
    fig.show()
    if save_fig:
        fig.write_image(f"{path_fig}/{path_plot}/borgatti-everett/{topic}.pdf")

##### Rombach

In [ ]:
import os

if not os.path.exists(f"{path_fig}/{path_plot}/rombach/"):
    os.mkdir(f"{path_fig}/{path_plot}/rombach/")

In [ ]:
for topic in pd.unique(comm_t.topic):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=comm_t[comm_t.topic==topic].year,
                             y=comm_t[comm_t.topic==topic].core_rb,
                             mode='markers',
                             marker={'size': 2}))
    fig.update_layout(template='plotly_white',
                      title_text=topic,
                      width=500, height=500,
                      xaxis={'title': 'year',
                             'range': [1000,2020]},
                      yaxis={'title': 'coreness'})
    fig.show()
    if save_fig:
        fig.write_image(f"{path_fig}/{path_plot}/rombach/{topic}.pdf")

##### Lead-lag

In [ ]:
core = pd.DataFrame()
for topic, network in networks.items():
    for node in network.graph.nodes:
        if network.graph.nodes[node]['core_be'] == 1:
            year = network.graph.nodes[node]['year']
            neighbors = list(network.graph.successors(node)) +\
                        list(network.graph.predecessors(node))
            periphery = [neighbor for neighbor in neighbors
                         if network.graph.nodes[neighbor]['core_be'] == 0]
            new_rows = pd.DataFrame([[topic, node, year, periphery]],
                                    columns=['topic','core','year','periphery'])
            core = pd.concat([core, new_rows], ignore_index=True)
core

In [ ]:
core.iloc[0].core, core.iloc[0].periphery

In [ ]:
[(node, networks['anatomy'].graph.nodes[node]['core_be'], networks['anatomy'].graph.nodes[node]['year'])
 for node in list(networks['anatomy'].graph.predecessors('Biology')) +\
             list(networks['anatomy'].graph.successors('Biology'))]

In [ ]:
core_expand = core.periphery\
                  .apply(pd.Series)\
                  .merge(core, left_index=True, right_index=True)\
                  .drop(['periphery'], axis=1)\
                  .melt(id_vars=['topic','core','year'], value_name='periphery')\
                  .drop('variable', axis=1)\
                  .sort_values(by=['topic','core'])\
                  .dropna()\
                  .reset_index(drop=True)
core_expand['year_periphery'] = [networks[core_expand.iloc[i].topic].graph\
                                     .nodes[core_expand.iloc[i].periphery]['year']
                                 for i in core_expand.index]
core_expand

In [ ]:
import os

if not os.path.exists(f"{path_fig}/{path_plot}/lead_lag"):
    os.mkdir(f"{path_fig}/{path_plot}/lead_lag")

In [ ]:
for topic in pd.unique(core_expand.topic):
    fig = go.Figure()
    data = core_expand[core_expand.topic==topic]
    fig.add_trace(go.Scatter(x=data.year,
                             y=data.year_periphery,
                             mode='markers',
                             marker={'size': 2}))
    fig.update_layout(template='plotly_white',
                      title_text=topic,
                      width=500, height=500,
                      xaxis={'title': 'year (core)',
                             'range': [np.min(data.year)-100, np.max(data.year)+100]},
                      yaxis={'title': 'year (neighboring periphery)',
                             'range': [np.min(data.year)-100, np.max(data.year)+100],
                             'scaleanchor': 'x',
                             'scaleratio': 1})
    fig.show()
    if save_fig:
        fig.write_image(f"{path_fig}/{path_plot}/lead_lag/{topic}.pdf")

##### Summary

In [ ]:
fig = go.Figure()
for topic in pd.unique(core_expand.topic):
    data = core_expand[core_expand.topic==topic]
    fig.add_trace(go.Violin(x=data.year-data.year_periphery,
                            name=topic,
                            marker={'size': 5,
                                    'symbol': 'line-ns-open',
                                    'color': '#2A3F5F'}))
fig.update_layout(template='plotly_white',
                  height=1000,
                  title_text='year (core) - year (periphery)',
                  showlegend=False)
fig.show()
if save_fig:
    fig.write_image(f"{path_fig}/{path_plot}/lead_lag.pdf")

## Modularity

In [ ]:
import os

path_plot = '1 modularity'

if not os.path.exists(f"{path_fig}/{path_plot}"):
    os.mkdir(f"{path_fig}/{path_plot}")

### Static

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_mean['modularity'],
                         y=df_mean['modularity-null-target'],
                         mode='markers',
                         line={'color': '#2A3F5F'},
                         showlegend=False,
                         hovertext=df_mean['topic']))
fig.add_trace(go.Scatter(x=[0,1], y=[0,1],
                         mode='lines',
                         line={'dash': 'dash',
                               'color': '#2A3F5F'},
                         showlegend=False))
fig.update_layout(template='plotly_white',
                  title='modularity',
                  width=500, height=500,
                  xaxis=dict(title='null',
                             range=[0,1]),
                  yaxis=dict(title='real',
                             range=[0,1],
                             scaleanchor='x',
                             scaleratio=1))
fig.show()
if save_fig:
    fig.write_image(f"{path_fig}/{path_plot}/modularity.pdf")

### Growth

In [ ]:
import os

if not os.path.exists(f"{path_fig}/{path_modularity}/growth/"):
    os.mkdir(f"{path_fig}/{path_modularity}/growth")

In [ ]:
for topic in networks.keys():
    fig = go.Figure()
    data = comm_t[comm_t.topic==topic]
    for i in sorted(pd.unique(comm_t.comm)):
        fig.add_trace(go.Scatter(x=data[data.comm==i]['year'],
                                 y=data[data.comm==i]['comm_count'],
                                 mode='lines',
                                 name=str(i)))
    fig.update_layout(template='plotly_white',
                      title_text=topic,
                      xaxis={'range': [0,2000],
                             'title': 'year'},
                      yaxis={'title': '# nodes',
                             'range': [-1,np.max(data.comm_count)]})
    fig.show()
    fig.write_image(f"{path_fig}/{path_modularity}/growth/{topic}.pdf")

### Cores in modules

In [ ]:
import os

if not os.path.exists(f"{path_fig}/{path_plot}/core_module/"):
    os.mkdir(f"{path_fig}/{path_plot}/core_module")

In [ ]:
comm_core = pd.concat([pd.DataFrame([[topic,
                                      node,
                                      network.graph.nodes[node]['year'],
                                      network.graph.nodes[node]['community'],
                                      network.graph.nodes[node]['community_core_be'],
                                      1 if network.graph.nodes[node]['community_core_be']==0 else 0,
                                      network.graph.graph['community_coreness_be']\
                                          [network.graph.nodes[node]['community']],
                                      1
                                     ]],
                                    columns=['topic','node','year','community','community_core',
                                             'community_peri','community_coreness','count'])
                       for topic, network in networks.items()
                       for node in network.graph.nodes],
                      ignore_index=True)\
              .sort_values(by='year')
comm_core = comm_core\
              .merge(comm_core.groupby(['community'])['count'].sum(),
                     on=['community'],
                     suffixes=('','_sum'))
comm_core['core_count'] = comm_core.groupby(['community'])['community_core']\
                                             .transform(pd.Series.cumsum)
comm_core['peri_count'] = comm_core.groupby(['community'])['community_peri']\
                                             .transform(pd.Series.cumsum)
comm_core = comm_core.drop(['count', 'count_sum', 'community_core', 'community_peri'], axis=1)
comm_core

In [ ]:
for topic, network in networks.items():
    fig = go.Figure()
    for i in range(4):
        data = comm_core[(comm_core.topic==topic) & (comm_core.community==i)]
        fig.add_trace(go.Scatter(x=data['year'],
                                 y=(data['core_count']/np.max(data['core_count']))\
                                     /(data['peri_count']/np.max(data['peri_count'])),
                                 mode='lines',
                                 name=f"community {i}"))
    fig.add_trace(go.Scatter(x=[np.min(comm_core.year),np.max(comm_core.year)],
                             y=[1,1],
                             mode='lines',
                             line={'dash': 'dash'},
                             name='equal'))
    fig.update_layout(template='plotly_white',
                      title_text=topic,
                      xaxis={'range': [1500,2030]},
                      yaxis={'title': '% cores/% periphery'},
                      height=400)
    fig.show()
    if save_fig:
        fig.write_image(f"{path_fig}/{path_plot}/core_module/{topic}.pdf")

### Summary

In [ ]:
core_mod = pd.DataFrame()
for topic, network in networks.items():
    for node in network.graph.nodes:
        if network.graph.nodes[node]['community_core_be'] == 1:
            year = network.graph.nodes[node]['year']
            community = network.graph.nodes[node]['community']
            neighbors = list(network.graph.successors(node)) +\
                        list(network.graph.predecessors(node))
            periphery = [neighbor for neighbor in neighbors
                         if network.graph.nodes[neighbor]['community'] == community and\
                            network.graph.nodes[neighbor]['community_core_be'] == 0]
            new_rows = pd.DataFrame([[topic, node, year, community, periphery]],
                                    columns=['topic','core','year','community','periphery'])
            core_mod = pd.concat([core_mod, new_rows], ignore_index=True)
core_mod

In [ ]:
core_mod_expand = core_mod.periphery\
                          .apply(pd.Series)\
                          .merge(core_mod, left_index=True, right_index=True)\
                          .drop(['periphery'], axis=1)\
                          .melt(id_vars=['topic','core','year','community'], value_name='periphery')\
                          .drop('variable', axis=1)\
                          .sort_values(by=['topic','core'])\
                          .dropna()\
                          .reset_index(drop=True)
core_mod_expand['year_periphery'] = [networks[core_mod_expand.iloc[i].topic].graph\
                                       .nodes[core_mod_expand.iloc[i].periphery]['year']
                                     for i in core_mod_expand.index]
core_mod_expand

In [ ]:
fig = go.Figure()
for topic in pd.unique(core_mod_expand.topic):
    data = core_mod_expand[core_mod_expand.topic==topic]
    fig.add_trace(go.Violin(x=data.year-data.year_periphery,
                            name=topic,
                            marker={'size': 5,
                                    'symbol': 'line-ns-open'}))
fig.update_layout(template='plotly_white',
                  height=1000,
                  title_text='year (core) - year (periphery)',
                  xaxis={},
                  showlegend=False)
fig.show()
if save_fig:
    fig.write_image(f"{path_fig}/{path_plot}/lead_lag.pdf")

**Note**: We're more explorers than formulists.